In [1]:
"""Plot STIM OFF vs. ON power (Figure 2A)."""

import os
import sys
from pathlib import Path
import numpy as np
import copy
import pandas as pd
import pte_stats
import pte_decode
from matplotlib import pyplot as plt

cd_path = Path(os.getcwd()).absolute().parent
sys.path.append(os.path.join(cd_path, "coherence"))

import matplotlib
matplotlib.rc('xtick', labelsize=6)
matplotlib.rc('ytick', labelsize=6)
matplotlib.rc('legend', fontsize=6)
matplotlib.rc("font", size=6, family="Arial")
matplotlib.rc('axes', labelsize=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

prop_cycle = plt.rcParams["axes.prop_cycle"]
def_colors = prop_cycle.by_key()["color"]

n_perm = 100_000
alpha = 0.05
two_sided = True

FOLDERPATH_ANALYSIS = "Path_to\\Project\\Analysis"
FOLDERPATH_FIGURES = os.path.join(os.path.dirname(os.getcwd()), "figures")

lfreq = 4.0
hfreq = 40.0

fbands = {
    "alpha": (8, 12),
    "low_beta": (12, 20),
    "high_beta": (20, 30)
}

fband_func = np.mean

In [ ]:
# WHOLE CORTEX POWER (Figure 2A)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_standard_whole-StimOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["freqs"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["freqs"] = [freqs] * len(data["ch_types"])
data = pd.DataFrame.from_dict(data)

pow_off = np.array(data["power-multitaper"][(data["stim"] == "Off") & (data["ch_regions"] == "cortex")].to_list())
pow_on = np.array(data["power-multitaper"][(data["stim"] == "On") & (data["ch_regions"] == "cortex")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

pow_off = pow_off[:, lfreq_i:hfreq_i+1]
pow_on = pow_on[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=pow_off.T,
    x_2=pow_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON STN-DBS"],
    x_label="Frequency (Hz)",
    y_label="Power (% total)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex",
    colour=[def_colors[2], def_colors[4]]
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(pow_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(pow_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0, zorder=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_box_aspect(0.8)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="lower left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.set_frame_on(False)
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.set_xticks(np.arange(5, 45, 5))
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1
axis.set_yticks([1, 2, 3])

fig.set_size_inches(1.6, 1.6)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_Stim_whole_cortex.pdf"), dpi=300)

In [ ]:
# STN POWER (Figure 2A)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_standard_whole-StimOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["freqs"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["freqs"] = [freqs] * len(data["ch_types"])
data = pd.DataFrame.from_dict(data)

pow_off = np.array(data["power-multitaper"][(data["stim"] == "Off") & (data["ch_regions"] == "STN")].to_list())
pow_on = np.array(data["power-multitaper"][(data["stim"] == "On") & (data["ch_regions"] == "STN")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

pow_off = pow_off[:, lfreq_i:hfreq_i+1]
pow_on = pow_on[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=pow_off.T,
    x_2=pow_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON STN-DBS"],
    x_label="Frequency (Hz)",
    y_label="Power (% total)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="STN",
    colour=[def_colors[2], def_colors[4]]
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(pow_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(pow_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=True
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0, zorder=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_box_aspect(0.8)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="lower left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.legend_handles[2].set_color([0.5, 0.5, 0.5])
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.set_ylim((0.2, 3.1))
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1
axis.set_yticks([1, 2, 3])

fig.set_size_inches(1.58, 1.58)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_Stim_STN.pdf"))